[Sascha Spors](https://orcid.org/0000-0001-7225-9992),
Professorship Signal Theory and Digital Signal Processing,
[Institute of Communications Engineering (INT)](https://www.int.uni-rostock.de/),
Faculty of Computer Science and Electrical Engineering (IEF),
[University of Rostock, Germany](https://www.uni-rostock.de/en/)

# Tutorial Digital Signal Processing (Digitale Signalverarbietung)

Winter Semester 2021/2022 (Bachelor Course #24505)

- lecture: https://github.com/spatialaudio/digital-signal-processing-lecture
- tutorial: https://github.com/spatialaudio/digital-signal-processing-exercises

WIP...
The project is currently under heavy development while adding new material for the winter semester 2021/2021

Feel free to contact lecturer [frank.schultz@uni-rostock.de](https://orcid.org/0000-0002-3010-0294)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import diric
import scipy.signal.windows as window
%matplotlib

This Code generates the graphics for https://github.com/spatialaudio/digital-signal-processing-exercises/blob/master/dft/dft_windowing_tutorial/dft_windowing_tutorial.tex and bases on its graphics/the matlab code which originally created the plots.

In [ ]:
N = 16
k = np.arange(0,N,1)
y1 = np.exp(2j*np.pi/N*k)
y2 = np.exp(2j*np.pi/N*2*k)
fig, (ax1,ax2) = plt.subplots(nrows=2,figsize=(10,5))
ax1.grid(True)
ax1.set_xticks(np.arange(0,N-1,2))
ax1.set_yticks(np.arange(-1,1.5,0.5))
ax1.set_ylim(-1.05,1.05)
ax1.set_xlim(0,N-1)
ax1.plot(k,np.real(y1),label="$\Re\{\mathbf{W}_{\mathrm{column2}}\}$",marker="o",ms=7.0)
ax1.plot(k,np.imag(y1),label="$\Im\{\mathbf{W}_{\mathrm{column2}}\}$",color="orange",marker="o",ms=7.0)
ax1.legend()
ax2.grid(True)
ax2.set_xticks(np.arange(0,N-1,2))
ax2.set_yticks(np.arange(-1,1.5,0.5))
ax2.set_ylim(-1.05,1.05)
ax2.set_xlim(0,N-1)
ax2.plot(k,np.real(y2),label="$\Re\{\mathbf{W}_{\mathrm{column3}}\}$",marker="o",ms=7.0)
ax2.plot(k,np.imag(y2),label="$\Im\{\mathbf{W}_{\mathrm{column3}}\}$",color="orange",marker="o",ms=7.0)
ax2.legend()
fig.savefig('graphics/dft_eigensignals_columns.pdf')

In [ ]:
def pcolor_mesh_fourier_matrix(N,ax,fig,abs=False):
    data = np.zeros((N,N))
    for i in range(0,N):
        for j in range(0,N):
            if abs is False:
                data[i,j]=np.angle(np.exp(2j*np.pi/N*i*j),deg=True)
            else:
                data[i,j]=np.abs(np.angle(np.exp(2j*np.pi/N*i*j),deg=True))
    fig.colorbar(ax.pcolormesh(data,edgecolors='none',cmap='RdBu',vmax=180,vmin=-180),ax=ax,ticks=np.linspace(start=-180,stop=180,num=9,endpoint=True))
    ticks = np.arange(start=0,stop=N,step=np.ceil(N/10),dtype=int)
    ax.set_xticks(ticks+0.5)
    ax.set_xticklabels(ticks)
    ax.set_yticks(ticks+0.5)
    ax.set_yticklabels(ticks)
    ax.set_ylabel('k')
    ax.set_xlabel('$\mu$')
    if abs is True:
        ax.set_title('$|\mathrm{arg}(W_N^{k\cdot\mu})|$ in $\circ$, $N$='+str(N))
    else:
        ax.set_title('$\mathrm{arg}(W_N^{k\cdot\mu})$ in $\circ$, $N$='+str(N))
    ax.set_aspect(aspect='equal',adjustable='box')

In [ ]:
def pcolor_mesh_fourier_matrix_abs_and_no_abs(N,path):
    fig, (ax1,ax2) = plt.subplots(ncols=2,figsize=(10,4))
    pcolor_mesh_fourier_matrix(N,ax1,fig,abs=False)
    pcolor_mesh_fourier_matrix(N,ax2,fig,abs=True)
    fig.savefig(path)

In [ ]:
pcolor_mesh_fourier_matrix_abs_and_no_abs(8,'graphics/TwiddleFactorMatrix_N8.pdf')
pcolor_mesh_fourier_matrix_abs_and_no_abs(9,'graphics/TwiddleFactorMatrix_N9.pdf')
pcolor_mesh_fourier_matrix_abs_and_no_abs(48,'graphics/TwiddleFactorMatrix_N48.pdf')

In [ ]:
def plot_dft_real_imag_mag_ph_periodicity(N,path):
    X = create_dft_for_plotting(N=N)
    k1 = np.arange(-N,0,1)
    k2 = np.arange(0,N,1)
    k3 = np.arange(N,2*N,1)
    fig, ((ax1,ax2),(ax3,ax4)) = plt.subplots(nrows=2,ncols=2,figsize=(10,6))
    #Plot the real part of X
    ax1.plot(k1,np.real(X),color='black')
    ax1.plot(k2,np.real(X),color='blue')
    ax1.plot(k3,np.real(X),color='black')
    ax1.scatter(k1,np.real(X),edgecolor='black',facecolor='none')
    ax1.scatter(k2,np.real(X),edgecolor='blue',facecolor='none')
    ax1.scatter(k3,np.real(X),edgecolor='black',facecolor='none')
    ax1.grid(True)
    ax1.set_ylabel('$\mathrm{Re}\{X[\mu]\}$')
    ax1.set_xlabel('$\mu$')
    ax1.set_xticks(np.arange(-N,2*N,np.ceil(N/4)))
    ax1.set_yticks(np.array([]))
    #Plot the imag part of X
    ax2.plot(k1,np.imag(X),color='black')
    ax2.plot(k2,np.imag(X),color='blue')
    ax2.plot(k3,np.imag(X),color='black')
    ax2.scatter(k1,np.imag(X),edgecolor='black',facecolor='none')
    ax2.scatter(k2,np.imag(X),edgecolor='blue',facecolor='none')
    ax2.scatter(k3,np.imag(X),edgecolor='black',facecolor='none')
    ax2.grid(True)
    ax2.set_ylabel('$\mathrm{Im}\{X[\mu]\}$')
    ax2.set_xlabel('$\mu$')
    ax2.set_xticks(np.arange(-N,2*N,np.ceil(N/4)))
    ax2.set_yticks(np.array([]))
    #Plot the magnitude of X
    ax3.plot(k1,np.abs(X),color='black')
    ax3.plot(k2,np.abs(X),color='blue')
    ax3.plot(k3,np.abs(X),color='black')
    ax3.scatter(k1,np.abs(X),edgecolor='black',facecolor='none')
    ax3.scatter(k2,np.abs(X),edgecolor='blue',facecolor='none')
    ax3.scatter(k3,np.abs(X),edgecolor='black',facecolor='none')
    ax3.grid(True)
    ax3.set_ylabel('$|X[\mu]|$')
    ax3.set_xlabel('$\mu$')
    ax3.set_xticks(np.arange(-N,2*N,np.ceil(N/4)))
    ax3.set_yticks(np.array([]))
    #Plot the phase of X
    ax4.plot(k1,np.angle(X),color='black')
    ax4.plot(k2,np.angle(X),color='blue')
    ax4.plot(k3,np.angle(X),color='black')
    ax4.scatter(k1,np.angle(X),edgecolor='black',facecolor='none')
    ax4.scatter(k2,np.angle(X),edgecolor='blue',facecolor='none')
    ax4.scatter(k3,np.angle(X),edgecolor='black',facecolor='none')
    ax4.grid(True)
    ax4.set_ylabel('$arg(X[\mu])$')
    ax4.set_xlabel('$\mu$')
    ax4.set_xticks(np.arange(-N,2*N,np.ceil(N/4)))
    ax4.set_yticks(np.array([]))
    fig.tight_layout()
    fig.savefig(path)
def create_dft_for_plotting(N=8):# PLEASE IMPLEMENT A FUNCTION TO GENERATE WELL LOOKING DFTs
    k = np.arange(0,N,1)
    x = np.cos(2*k)+(k-0.5)*3
    return np.fft.fft(x)

In [ ]:
plot_dft_real_imag_mag_ph_periodicity(8,'graphics/Periodicity_DFT.pdf')

In [ ]:
def plot_dft_real_imag_mag_ph_symmetrie(N,path=None):
    X = create_dft_for_plotting(N=N)
    k = np.arange(0,N+1,1)
    fig, ((ax1,ax2),(ax3,ax4))= plt.subplots(2,2,figsize=(10,6))
    fig.tight_layout()
    #Plot real part of X
    ax1.plot(k,np.append(np.real(X),np.real(X[0])),color='blue')
    ax1.scatter(k,np.append(np.real(X),np.real(X[0])),edgecolor='blue',facecolor='none')
    ax1.set_ylabel('$\mathrm{Re}\{X[\mu]\}$')
    ax1.set_xlabel('$\mu$')
    d = np.abs(np.amax(np.real(X))-np.amin(np.real(X)))
    if d == 0:
        d = 0.1
    ax1.set_xticks(np.arange(0,N+1,np.ceil(N/10)))
    ax1.set_yticks(np.array([]))
    ax1.set_ylim(np.amin(np.real(X))-1/5*d,np.amax(np.real(X))+1/5*d)
    ax1.plot([N/2,N/2],[np.amin(np.real(X))-d,np.amax(np.real(X))+d])
    #Plot imag part of X
    ax2.plot(k,np.append(np.imag(X),np.imag(X[0])),color='blue')
    ax2.scatter(k,np.append(np.imag(X),np.imag(X[0])),edgecolor='blue',facecolor='none')
    ax2.set_ylabel('$\mathrm{Im}\{X[\mu]\}$')
    ax2.set_xlabel('$\mu$')
    d = np.abs(np.amax(np.imag(X))-np.amin(np.imag(X)))
    if d == 0:
        d = 0.1
    ax2.set_xticks(np.arange(0,N+1,np.ceil(N/10)))
    ax2.set_yticks(np.array([]))
    ax2.set_ylim(np.amin(np.imag(X))-1/5*d,np.amax(np.imag(X))+1/5*d)
    if N % 2 == 0:
        ax2.scatter(N/2,np.imag(X[int(N/2)]),edgecolor='none',facecolor='red')
    else:
        ax2.scatter(N/2,(np.imag(X[int(N/2)])+np.imag(X[int(N/2)+1]))/2,edgecolor='none',facecolor='red')
    #Plot the magnitude of X
    ax3.plot(k,np.append(np.abs(X),np.abs(X[0])),color='blue')
    ax3.scatter(k,np.append(np.abs(X),np.abs(X[0])),edgecolor='blue',facecolor='none')
    ax3.set_ylabel('$|X[\mu]|$')
    ax3.set_xlabel('$\mu$')
    d = np.abs(np.amax(np.abs(X))-np.amin(np.abs(X)))
    if d == 0:
        d = 0.1
    ax3.set_xticks(np.arange(0,N+1,np.ceil(N/10)))
    ax3.set_yticks(np.array([]))
    ax3.set_ylim(np.amin(np.abs(X))-1/5*d,np.amax(np.abs(X))+1/5*d)
    ax3.plot([N/2,N/2],[np.amin(np.abs(X))-d,np.amax(np.abs(X))+d])
    #Plot the phase of X
    ax4.plot(k,np.append(np.angle(X),np.angle(X[0])),color='blue')
    ax4.scatter(k,np.append(np.angle(X),np.angle(X[0])),edgecolor='blue',facecolor='none')
    ax4.set_ylabel('$\mathrm{arg}({X[\mu])$')
    ax4.set_xlabel('$\mu$')
    d = np.abs(np.amax(np.angle(X))-np.amin(np.angle(X)))
    if d == 0:
        d = 0.1
    ax4.set_xticks(np.arange(0,N+1,np.ceil(N/10)))
    ax4.set_yticks(np.array([]))
    ax4.set_ylim(np.amin(np.angle(X))-1/5*d,np.amax(np.angle(X))+1/5*d)
    if N % 2 == 0:
        ax4.scatter(N/2,np.angle(X[int(N/2)]),edgecolor='none',facecolor='red')
    else:
        ax4.scatter(N/2,(np.angle(X[int(N/2)])+np.angle(X[int(N/2)+1]))/2,edgecolor='none',facecolor='red')
    fig.savefig(path)

In [ ]:
plot_dft_real_imag_mag_ph_symmetrie(N=8,path='graphics/Symmetry_DFT_N8.pdf')
plot_dft_real_imag_mag_ph_symmetrie(N=9,path='graphics/Symmetry_DFT_N9.pdf')

In [ ]:
fig, (ax1,ax2) = plt.subplots(nrows=1,ncols=2,figsize=(10,4))
N=4
t = np.arange(0,2*np.pi,1/100)
ax1.plot(np.cos(t),np.sin(t),color='black')
ax1.set_xlim(-1.05,1.05)
ax1.set_ylim(-1.05,1.05)
ax1.set_xticks(np.arange(-1,1.5,0.5))
ax1.set_yticks(np.arange(-1,1.5,0.5))
ax1.set_aspect(aspect='equal',adjustable='box')
ax1.set_title('Eigenfrequencies of DFT, N = '+str(N))
k = np.arange(0,N,1)
ax1.scatter(np.cos(2*np.pi/N*k),np.sin(2*np.pi/N*k),edgecolor='blue',facecolor='none')
ax1.text(x=0.75,y=0,s='$\Omega=0$\n$f=0\cdot f_s$',ha='center',va='center',bbox=dict(ec='black',fc='none'))
ax1.text(x=-0.75,y=0,s=r'$\Omega=\pi$' '\n' r'$f=\frac{f_s}{2}$',ha='center',va='center',bbox=dict(ec='black',fc='none'))
ax1.text(x=0,y=0.75,s=r'$\Omega=\frac{2\pi}{N}$',ha='center',va='center',bbox=dict(ec='black',fc='none'))
ax1.text(x=0,y=-0.75,s=r'$\Omega=\frac{2\pi(N-1)}{N}$',ha='center',va='center',bbox=dict(ec='black',fc='none'))
N=5
ax2.plot(np.cos(t),np.sin(t),color='black')
ax2.set_xlim(-1.05,1.05)
ax2.set_ylim(-1.05,1.05)
ax2.set_xticks(np.arange(-1,1.5,0.5))
ax2.set_yticks(np.arange(-1,1.5,0.5))
ax2.set_aspect(aspect='equal',adjustable='box')
ax2.set_title('Eigenfrequencies of DFT, N = '+str(N))
k = np.arange(0,N,1)
ax2.scatter(np.cos(2*np.pi/N*k),np.sin(2*np.pi/N*k),edgecolor='blue',facecolor='none')
ax2.text(x=0.75,y=0,s='$\Omega=0$\n$f=0\cdot f_s$',ha='center',va='center',bbox=dict(ec='black',fc='none'))
ax2.text(x=-0.75,y=0,s=r'$\Omega=\pi$' '\n' r'$f=\frac{f_s}{2}$',ha='center',va='center',bbox=dict(ec='black',fc='none'))
ax2.text(x=0.2,y=0.75,s=r'$\Omega=\frac{2\pi}{N}$',ha='center',va='center',bbox=dict(ec='black',fc='none'))
ax2.text(x=0.2,y=-0.75,s=r'$\Omega=\frac{2\pi(N-1)}{N}$',ha='center',va='center',bbox=dict(ec='black',fc='none'))
fig.tight_layout()
fig.savefig('graphics/DFT_Eigenfrequencen_N4_N5')

In [ ]:
fig, (ax1,ax2) = plt.subplots(nrows=1,ncols=2,figsize=(10,4))
N=8
t = np.arange(0,2*np.pi,1/100)
ax1.plot(np.cos(t),np.sin(t),color='black')
ax1.set_xlim(-1.05,1.05)
ax1.set_ylim(-1.05,1.05)
ax1.set_xticks(np.arange(-1,1.5,0.5))
ax1.set_yticks(np.arange(-1,1.5,0.5))
ax1.set_aspect(aspect='equal',adjustable='box')
ax1.set_title('Eigenfrequencies of DFT, N = '+str(N))
k = np.arange(0,N,1)
ax1.scatter(np.cos(2*np.pi/N*k),np.sin(2*np.pi/N*k),edgecolor='blue',facecolor='none')
ax1.text(x=0.75,y=0,s='$\Omega=0$\n$f=0\cdot f_s$',ha='center',va='center',bbox=dict(ec='black',fc='none'))
ax1.text(x=-0.75,y=0,s=r'$\Omega=\pi$' '\n' r'$f=\frac{f_s}{2}$',ha='center',va='center',bbox=dict(ec='black',fc='none'))
ax1.text(x=0.5,y=0.5,s=r'$\Omega=\frac{2\pi}{N}$',ha='center',va='center',bbox=dict(ec='black',fc='none'))
ax1.text(x=0.5,y=-0.5,s=r'$\Omega=\frac{2\pi(N-1)}{N}$',ha='center',va='center',bbox=dict(ec='black',fc='none'))
N=9
ax2.plot(np.cos(t),np.sin(t),color='black')
ax2.set_xlim(-1.05,1.05)
ax2.set_ylim(-1.05,1.05)
ax2.set_xticks(np.arange(-1,1.5,0.5))
ax2.set_yticks(np.arange(-1,1.5,0.5))
ax2.set_aspect(aspect='equal',adjustable='box')
ax2.set_title('Eigenfrequencies of DFT, N = '+str(N))
k = np.arange(0,N,1)
ax2.scatter(np.cos(2*np.pi/N*k),np.sin(2*np.pi/N*k),edgecolor='blue',facecolor='none')
ax2.text(x=0.75,y=0,s='$\Omega=0$\n$f=0\cdot f_s$',ha='center',va='center',bbox=dict(ec='black',fc='none'))
ax2.text(x=-0.75,y=0,s=r'$\Omega=\pi$' '\n' r'$f=\frac{f_s}{2}$',ha='center',va='center',bbox=dict(ec='black',fc='none'))
ax2.text(x=0.5,y=0.5,s=r'$\Omega=\frac{2\pi}{N}$',ha='center',va='center',bbox=dict(ec='black',fc='none'))
ax2.text(x=0.5,y=-0.5,s=r'$\Omega=\frac{2\pi(N-1)}{N}$',ha='center',va='center',bbox=dict(ec='black',fc='none'))
fig.tight_layout()
fig.savefig('graphics/DFT_Eigenfrequencen_N8_N9')

In [ ]:
def create_DFT_for_interpol_plot(N):
    k = np.arange(0,N,1)
    x = 0.5+0.8*np.sin(np.pi/4*k)+0.4*np.sin(np.pi/2*k)+1.5*np.sin(3*np.pi/4*k)+0.75/2*np.sin(np.pi*k-np.pi/2)
    return k, np.fft.fft(x)
def interpolate_DFT(X,Omega_int):
    N = X.size
    X_int = np.zeros(Omega_int.size,dtype=complex)
    for n in range(0,Omega_int.size):
        for mu in range(0,N):
            tmp = Omega_int[n]-2*np.pi/N*mu
            X_int[n]=X_int[n]+X[mu]*np.exp(-1j*tmp/2*(N-1))*diric(tmp,N)
    return X_int
def plot_interpolation_DFT(N,ax,mode='mu',setTitle=False):
    mu, X = create_DFT_for_interpol_plot(N)
    Omega_int=np.arange(0,2*np.pi,1/128*np.pi)
    X_int = interpolate_DFT(X=X,Omega_int=Omega_int)
    if mode == 'mu':
        ax.stem(mu,np.abs(X)/N,basefmt=" ")
        ax.scatter(N,np.abs(X[0])/N,color='black')
        ax.set_xlim(0,N)
        ax.set_xticks(np.arange(0,N,np.ceil(N/10)))
        ax.plot(N*Omega_int/(2*np.pi),np.abs(X_int)/N,color='C1')
        ax.set_xlabel('$\mu$')
    elif mode == 'pi':
        ax.stem(mu/N*2,np.abs(X)/N,basefmt=" ")
        ax.scatter(2,np.abs(X[0])/N,color='black')
        ax.set_xlim(0,2)
        ax.set_xticks(np.arange(0,2.1,0.5))
        ax.plot(Omega_int/(np.pi),np.abs(X_int)/N,color='C1')
        ax.set_xlabel('$\Omega\enspace /\enspace \pi$')
    elif mode == 'fs':
        ax.stem(mu/N,np.abs(X)/N,basefmt=" ")
        ax.scatter(1,np.abs(X[0])/N,color='black')
        ax.set_xlim(0,1)
        ax.set_xticks(np.arange(0,1.1,0.25))
        ax.plot(Omega_int/(2*np.pi),np.abs(X_int)/N,color='C1')
        ax.set_xlabel('$f\enspace /\enspace f_s$')
    else:
        plot_interpolation_DFT(N,ax,mode='mu')
        return
    ax.set_ylabel('$|X[\mu]|,|X(\Omega)|$')
    ax.set_ylim(0,1)
    ax.set_yticks(np.arange(0,1.1,0.5))
    ax.grid(True)
    if setTitle is True:
        ax.set_title('$N=%d$' %(N))

In [ ]:
fig, ((ax1,ax2),(ax3,ax4),(ax5,ax6)) = plt.subplots(nrows=3,ncols=2,figsize=(10,10))
plot_interpolation_DFT(N=8,ax=ax1,mode='mu',setTitle=True)
plot_interpolation_DFT(N=9,ax=ax2,mode='mu',setTitle=True)
plot_interpolation_DFT(N=8,ax=ax3,mode='pi',setTitle=False)
plot_interpolation_DFT(N=9,ax=ax4,mode='pi',setTitle=False)
plot_interpolation_DFT(N=8,ax=ax5,mode='fs',setTitle=False)
plot_interpolation_DFT(N=9,ax=ax6,mode='fs',setTitle=False)
fig.tight_layout()
fig.savefig('graphics/DTFT_Interpolation.pdf')

In [ ]:
def plot_dtft_dft_rectangular_window_spectrum_asym(ax,N=1,ylim=6,mode='lin'):
    """IN LOG MOD THIS FUNCTION SHOULD ONLY BE USED WITH N = 1 AND 100"""
    Omega = np.arange(1/128*np.pi,2*np.pi,1/128*np.pi)
    X_dtft = np.exp(-1j*Omega/2*(N-1))*np.sin(N*Omega/2)/np.sin(Omega/2)
    X_dft = np.zeros(N)
    X_dft[0] = N
    if mode == 'lin':
        ax.plot(np.append(np.array([0]),Omega)/np.pi,np.append(np.array([N]),np.abs(X_dtft)),color='red',label='DTFT')
        ax.scatter(np.arange(0,N,1)/N*2,np.abs(X_dft),color='blue',label='DFT')
        ax.set_ylabel('$|A|$')
        ax.set_yticks(np.arange(0,N+2,np.ceil(N/10)))
        ax.legend()
    elif mode == 'log':
        ax.plot(np.append(np.array([0]),Omega)/np.pi,20*np.log10(np.append(np.array([N]),np.abs(X_dtft))),color='black')
        ax.set_ylabel('$A\enspace/\enspace\mathrm{dB}$')
        y = 20*np.log10(N)
        if y < 20:
            y = 20
        else:
            y = int(y/5)*5+5
        ax.set_ylim(-20,y)
        ax.set_yticks(np.arange(-20,y+5,5))
        ax.set_xlim(0,2)
    ax.set_xticks(np.arange(0,2.1,0.5))
    ax.set_xlabel('$\Omega\enspace/\enspace\pi$')
    ax.set_title(r'$N=%d$' %(N))
    ax.grid(True)

In [ ]:
def plot_dtft_dft_rectangular_window_spectrum_sym(ax,N=1,ylim=6,mode='lin'):
    """IN LOG MOD THIS FUNCTION SHOULD ONLY BE USED WITH N = 1 AND 100"""
    Omega1 = np.arange(-np.pi,0,1/128*np.pi)
    Omega2 = np.arange(1/128*np.pi,np.pi,1/128*np.pi)
    Omega = np.append(Omega1,np.array([0]))
    Omega = np.append(Omega,Omega2)
    X_dtft1 = np.exp(-1j*Omega1/2*(N-1))*np.sin(N*Omega1/2)/np.sin(Omega1/2)
    X_dtft2 = np.exp(-1j*Omega2/2*(N-1))*np.sin(N*Omega2/2)/np.sin(Omega2/2)
    X_dtft1 = np.append(X_dtft1,np.array([N]))
    X_dtft1 = np.append(X_dtft1,X_dtft2)
    X_dft = np.zeros(N)
    X_dft[int(N/2)] = N
    if mode == 'lin':
        ax.plot(Omega/np.pi,np.abs(X_dtft1),color='red',label='DTFT')
        ax.scatter((np.arange(0,N,1)-int(N/2))/N*2,np.abs(X_dft),color='blue',label='DFT')
        ax.set_ylabel('$|A|$')
        ax.set_yticks(np.arange(0,N+2,np.ceil(N/10)))
        ax.legend()
    elif mode == 'log':
        ax.plot(Omega/np.pi,20*np.log10(np.abs(X_dtft1)),color='black')
        ax.set_ylabel('$A\enspace/\enspace\mathrm{dB}$')
        y = 20*np.log10(N)
        if y < 20:
            y = 20
        else:
            y = int(y/5)*5+5
        ax.set_ylim(-20,y)
        ax.set_yticks(np.arange(-20,y+5,5))
        ax.set_xlim(-1,1)
    ax.set_xticks(np.arange(-1,1.1,0.5))
    ax.set_xlabel('$\Omega\enspace/\enspace\pi$')
    ax.set_title(r'$N=%d$' %(N))
    ax.grid(True)

In [ ]:
fig, (ax1,ax2) = plt.subplots(ncols=2,nrows=1,figsize=(10,3.5))
plot_dtft_dft_rectangular_window_spectrum_asym(N=4,ax=ax1)
plot_dtft_dft_rectangular_window_spectrum_asym(N=5,ax=ax2)
fig.tight_layout()
fig.savefig('graphics/RectWindow_DTFT_DFT_lin_asym.pdf')

In [ ]:
fig, (ax1,ax2) = plt.subplots(ncols=2,nrows=1,figsize=(10,3.5))
plot_dtft_dft_rectangular_window_spectrum_sym(N=4,ax=ax1)
plot_dtft_dft_rectangular_window_spectrum_sym(N=5,ax=ax2)
fig.tight_layout()
fig.savefig('graphics/RectWindow_DTFT_DFT_lin_sym.pdf')

In [ ]:
fig, (ax1,ax2) = plt.subplots(ncols=2,nrows=1,figsize=(10,3.5))
plot_dtft_dft_rectangular_window_spectrum_asym(N=4,ax=ax1,mode='log')
plot_dtft_dft_rectangular_window_spectrum_asym(N=5,ax=ax2,mode='log')
fig.tight_layout()
fig.savefig('graphics/RectWindow_DTFT_DFT_log_asym.pdf')

In [ ]:
fig, (ax1,ax2) = plt.subplots(ncols=2,nrows=1,figsize=(10,3.5))
plot_dtft_dft_rectangular_window_spectrum_sym(N=4,ax=ax1,mode='log')
plot_dtft_dft_rectangular_window_spectrum_sym(N=5,ax=ax2,mode='log')
fig.tight_layout()
fig.savefig('graphics/RectWindow_DTFT_DFT_log_sym.pdf')

In [ ]:
fig, ((ax1,ax2),(ax3,ax4))= plt.subplots(ncols=2,nrows=2,figsize=(10,6))
k = np.arange(0,8)
A=10
N=8
x1 = A*np.exp(1j*np.pi*k/2)
x2 = A*np.exp(1j*np.pi/4*(2-1/2)*k)
X1 = np.fft.fft(x1)
X2 = np.fft.fft(x2)
ax1.stem(k,np.real(x1),basefmt='')
ax1.set_title(r'$x_1=%d \mathrm{exp}({\mathrm{j}\frac{2\pi}{8}\cdot 2k})$' %(A))
ax1.set_ylabel('$x_1[k]$')
ax1.set_xlabel('$k$')
ax1.set_ylim(-10.3,10.3)
ax1.set_xlim(-0.05,7.05)
ax1.set_xticks(np.arange(0,8,2))
ax1.set_yticks(np.arange(-10,11,5))
ax2.stem(k,np.real(x2),basefmt='')
ax2.set_title(r'$x_2=%d \mathrm{exp}({\mathrm{j}\frac{2\pi}{8}\cdot (2-\frac{1}{2})k})$' %(A))
ax2.set_ylabel('$x_2[k]$')
ax2.set_xlabel('$k$')
ax2.set_ylim(-10.3,10.3)
ax2.set_xlim(-0.05,7.05)
ax2.set_xticks(np.arange(0,8,2))
ax2.set_yticks(np.arange(-10,11,5))
ax3.stem(k,np.abs(X1)/N,basefmt='')
ax3.set_title('DFT of $x_1$ with $N=8$')
ax3.set_ylabel(r'$\frac{|X_1[\mu]|}{N}$')
ax3.set_xlabel('$\mu$')
ax3.set_ylim(0,10.3)
ax3.set_xlim(-0.05,7.05)
ax3.set_xticks(np.arange(0,8,2))
ax3.set_yticks(np.arange(0,11,2))
ax4.stem(k,np.abs(X2)/N,basefmt='')
ax4.set_title('DFT of $x_2$ with $N=8$')
ax4.set_ylabel(r'$\frac{|X_2[\mu]|}{N}$')
ax4.set_xlabel('$\mu$')
ax4.set_ylim(0,10.3)
ax4.set_xlim(-0.05,7.05)
ax4.set_xticks(np.arange(0,8,2))
ax4.set_yticks(np.arange(0,11,2))
fig.tight_layout()
#fig.savefig('graphics/DFTbestworstcase_RectWin.pdf')

In [ ]:
def hann_window_definitions(ax,N=8):
    """
    YOU SHOULD DEFINE A NEW UPPER BOUND FOR XTICKS FOR N>100
    """
    ax.set_title('Hann window definitions, $N=%d$' %(N))
    ax.set_ylabel('$w[k]$')
    ax.set_xlabel('$k$')
    k = np.arange(0,N)
    #for Hann window see more in https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.windows.hann.html#scipy.signal.windows.hann, last access 22.11.2021
    periodic = window.hann(M=N,sym=False)
    symmetric = window.windows.hann(M=N,sym=True)
    moeser = 0.5-0.5*np.cos(2*np.pi/N*(k+1/2))
    if N <= 20:
        ax.plot(k,periodic,color = 'red', ls = '-.', marker='o', mec= 'red',mfc = 'none',label='Hann Window, periodic')
        ax.plot(k,symmetric,color = 'black', ls = '-', marker='o', mec= 'black',mfc = 'none',label='Hann Window, symmetric')
        ax.plot(k,moeser,color = 'blue', ls = '--', marker='o', mec= 'blue',mfc = 'none',label='Möser, Ifeachor')
        ax.set_yticks(np.arange(0,1.5,0.2))
        ax.set_xticks(np.arange(0,N,2))
    else:
        ax.plot(k,periodic,color = 'red', ls = '-.', label='Hann Window, periodic')
        ax.plot(k,symmetric,color = 'black', ls = '-', label='Hann Window, symmetric')
        ax.plot(k,moeser,color = 'blue', ls = '--', label='Möser, Ifeachor')
        ax.set_yticks(np.arange(0,1.5,0.2))
        ax.set_xticks(np.arange(0,N,10))
    ax.legend()
    ax.grid(True)
    ax.set_ylim(0,1.4)
    ax.set_xlim(0,N-1)

In [ ]:
fig, (ax1,ax2) = plt.subplots(ncols = 2, nrows = 1, figsize= (10,4))
hann_window_definitions(ax1,N=8)
hann_window_definitions(ax2,N=32)
fig.tight_layout()
#fig.savefig('graphics/HannDefinitionen.pdf')

In [ ]:
def plot_hann_rect_window(N,ax,mode='normal'):
    omega1 = np.arange(-np.pi,0,1/1000*np.pi)
    omega2 = np.arange(0+1/1000*np.pi,np.pi,1/1000*np.pi)
    omega = np.append(np.append(omega1,np.zeros(1)),omega2)
    R11 = np.sin(N*omega1/2)/np.sin(omega1/2)
    R12 = np.sin(N*omega2/2)/np.sin(omega2/2)
    R1 = np.append(np.append(R11,np.array([N])),R12)
    R21 = 1/2*np.sin(N/2*(omega1-2*np.pi/N))/np.sin(1/2*(omega1-2*np.pi/N))
    R22 = 1/2*np.sin(N/2*(omega2-2*np.pi/N))/np.sin(1/2*(omega2-2*np.pi/N))
    #lim N->0 sin(-N/2*2*pi/N)/sin(-1/2*2*pi/N)=lim N->0 sin(-pi)/sin(-pi/n)= L'Hospital lim N->0 -pi*cos(-pi)/(-pi/N*cos(-pi/n))=lim N->0 N*1 = 0
    R2 = np.append(np.append(R21,np.zeros(1)),R22)
    R31 = 1/2*np.sin(N/2*(omega1+2*np.pi/N))/np.sin(1/2*(omega1+2*np.pi/N))
    R32 = 1/2*np.sin(N/2*(omega2+2*np.pi/N))/np.sin(1/2*(omega2+2*np.pi/N))
    #lim N->0 sin(N/2*2*pi/N)/sin(1/2*2*pi/N)=lim N->0 sin(pi)/sin(pi/n)= L'Hospital lim N->0 pi*cos(pi)/(pi/N*cos(pi/n))=lim N->0 N*1 = 0
    R3 = np.append(np.append(R31,np.zeros(1)),R32)
    if mode == 'normal':
        ax.plot(omega/np.pi,np.real(R1),color='black',label = 'rectangular window $R_1$')
        ax.plot(omega/np.pi,np.real(R2),color ='blue', ls='--',label = '$R_2$')
        ax.plot(omega/np.pi,np.real(R3),color='aqua',label='$R_3$')
        ax.plot(omega/np.pi,np.real((R1+R2+R3)),color = 'red',ls ='-.',label='Hann window $R_1+R_2+R_3$')
        ax.legend()
        mmin = min(np.amin(np.real(R1)),np.amin(np.real(R2)),np.amin(np.real(R3)),np.amin(np.real(R1+R2+R3)))
        mmax = N+1
        mmin = mmin - mmin % 1
        ax.set_ylim(mmin,mmax)
        ax.set_yticks(np.arange(mmin-(mmax-mmin)%np.ceil(N/10),mmax,np.ceil(N/10)))
        ax.set_ylabel('A')
    elif mode == 'lin':
        ax.plot(omega/np.pi,np.abs(R1)/N,color='black',label = 'rectangular window')
        #ax.plot(omega/np.pi,np.real(R2),color ='blue', ls='--',label = '$R_2$')
        #ax.plot(omega/np.pi,np.real(R3),color='aqua',label='$R_3$')
        ax.plot(omega/np.pi,np.abs(R1+R2+R3)/N,color = 'red',ls ='-.',label='Hann window')
        ax.legend()
        ax.set_ylim(-0.1,1.1)
        ax.set_yticks(np.arange(0,1.1,0.1))
        ax.set_ylabel(r'$\frac{|A|}{N}$')
    elif mode == 'log':
        ax.plot(omega/np.pi,20*np.log10(np.abs(R1)/N),color='black',label = 'rectangular window')
        #ax.plot(omega/np.pi,np.real(R2),color ='blue', ls='--',label = '$R_2$')
        #ax.plot(omega/np.pi,np.real(R3),color='aqua',label='$R_3$')
        ax.plot(omega/np.pi,20*np.log10(np.abs(R1+R2+R3)/N),color = 'red',ls ='-.',label='Hann window')
        ax.legend()
        ax.set_ylim(-60,0)
        ax.set_yticks(np.append(np.arange(-60,-6.1,6),np.arange(-6,0.1,3)))
        ax.set_ylabel(r'$20*\log_{10}\left(\frac{|A|}{N}\right)$')
    if (mode == 'normal' or mode =='lin') and N <=25:
        if N % 2 == 0:
            omega = np.arange(-N//2-1,N//2)*2*np.pi/N
        else:
            omega = np.arange(-N//2,N//2+1)*2*np.pi/N
        omega[N//2+1]=10**(-16)
        ax.plot(omega[1:N//2]/np.pi,np.zeros(len(omega[1:N//2])),ls='none',marker='o',color='red')
        ax.plot([omega[N//2]/np.pi,omega[N//2+2]/np.pi],np.zeros(2),ls='none',marker='o',color='black')
        if mode == 'normal':
            ax.plot(0,N,ls='none',marker='o',color='red')
        else:
            ax.plot(0,1,ls='none',marker='o',color='red')
        ax.plot(omega[N//2+3:]/np.pi,np.zeros(len(omega[N//2+3:])),ls='none',marker='o',color='red')
    ax.set_xlim(-1,1)
    ax.set_xticks(np.arange(-1,1.1,0.2))
    ax.set_title('Hann window, $N=%d$' %(N))
    ax.grid(True)
    ax.set_xlabel('$\Omega\enspace/\enspace\pi$')        

In [ ]:
fig, ax = plt.subplots(nrows = 1,ncols=1,figsize=(10,5))
plot_hann_rect_window(N=16,ax=ax,mode='normal')
fig.tight_layout()
#fig.savefig('graphics/HanningausRectWindow.pdf')
fig, ax = plt.subplots(nrows = 1,ncols=1,figsize=(10,5))
plot_hann_rect_window(N=16,ax=ax,mode='lin')
fig.tight_layout()
#fig.savefig('graphics/DTFTHanningWin_lin.pdf')
fig, ax = plt.subplots(nrows = 1,ncols=1,figsize=(10,5))
plot_hann_rect_window(N=16,ax=ax,mode='log')
fig.tight_layout()
#fig.savefig('graphics/DTFTHanningWin_log.pdf')

In [ ]:
N=8
k = np.arange(0,N,1)
x1 = 10*np.exp(1j*np.pi/2*k)
x2 = 10*np.exp(1j*np.pi/4*(2-1/2)*k)
whann = 1-np.cos(2*np.pi/N*(k+1/2))
X1 = np.fft.fft(x1*whann)
X2 = np.fft.fft(x2*whann)
fig, ((ax1,ax2),(ax3,ax4)) = plt.subplots(ncols=2,nrows=2,figsize=(10,6.5))
ax1.stem(k,np.real(x1),basefmt='black')
ax1.set_xticks(np.arange(0,N,2))
ax1.set_yticks(np.arange(-10,11,5))
ax1.grid(True)
ax1.set_title(r'$10\mathrm{exp}\left(\frac{2\pi}{8}\cdot 2k\right)$')
ax1.set_xlabel('$k$')
ax1.set_ylabel('$\Re\{x_1[k]}$')
ax2.stem(k,np.real(x2),basefmt='black')
ax2.set_xticks(np.arange(0,N,2))
ax2.set_yticks(np.arange(-10,11,5))
ax2.grid(True)
ax2.set_title(r'$10\mathrm{exp}\left(\frac{2\pi}{8}\cdot \left(2-\frac{1}{2}\right)k\right)$')
ax2.set_xlabel('$k$')
ax2.set_ylabel('$\Re\{x_2[k]\}$')
ax3.stem(k,np.abs(X1)/N,basefmt='black')
ax3.set_xticks(np.arange(0,N,2))
ax3.set_yticks(np.arange(0,11,2))
ax3.grid(True)
ax3.set_title('DFT of Hann-windowed $x_1$ with $N=%d$' %(N))
ax3.set_xlabel('$\mu$')
ax3.set_ylabel(r'$\frac{|X_1[\mu]|}{N}$')
ax4.stem(k,np.abs(X2)/N,basefmt='black')
ax4.set_xticks(np.arange(0,N,2))
ax4.set_yticks(np.arange(0,11,2))
ax4.grid(True)
ax4.set_title('DFT of Hann-windowed $x_2$ with $N=%d$' %(N))
ax4.set_ylabel(r'$\frac{|X_2[\mu]|}{N}$')
fig.tight_layout()
#fig.savefig('gaphics/DFTbestworstcase_HannWin.pdf')

In [ ]:
def plot_hamming_rect_window(N,ax,mode='normal'):
    beta = 21/25
    omega1 = np.arange(-np.pi,0,1/1000*np.pi)
    omega2 = np.arange(0+1/1000*np.pi,np.pi,1/1000*np.pi)
    omega = np.append(np.append(omega1,np.zeros(1)),omega2)
    R11 = np.sin(N*omega1/2)/np.sin(omega1/2)
    R12 = np.sin(N*omega2/2)/np.sin(omega2/2)
    R1 = np.append(np.append(R11,np.array([N])),R12)
    R21 = 1/2*np.sin(N/2*(omega1-2*np.pi/N))/np.sin(1/2*(omega1-2*np.pi/N))
    R22 = 1/2*np.sin(N/2*(omega2-2*np.pi/N))/np.sin(1/2*(omega2-2*np.pi/N))
    #lim N->0 sin(-N/2*2*pi/N)/sin(-1/2*2*pi/N)=lim N->0 sin(-pi)/sin(-pi/n)= L'Hospital lim N->0 -pi*cos(-pi)/(-pi/N*cos(-pi/n))=lim N->0 N*1 = 0
    R2 = beta*np.append(np.append(R21,np.zeros(1)),R22)
    R31 = 1/2*np.sin(N/2*(omega1+2*np.pi/N))/np.sin(1/2*(omega1+2*np.pi/N))
    R32 = 1/2*np.sin(N/2*(omega2+2*np.pi/N))/np.sin(1/2*(omega2+2*np.pi/N))
    #lim N->0 sin(N/2*2*pi/N)/sin(1/2*2*pi/N)=lim N->0 sin(pi)/sin(pi/n)= L'Hospital lim N->0 pi*cos(pi)/(pi/N*cos(pi/n))=lim N->0 N*1 = 0
    R3 = beta*np.append(np.append(R31,np.zeros(1)),R32)
    if mode == 'normal':
        ax.plot(omega/np.pi,np.real(R1),color='black',label = 'rectangular window $R_1$')
        ax.plot(omega/np.pi,np.real(R2),color ='blue', ls='--',label = '$R_2$')
        ax.plot(omega/np.pi,np.real(R3),color='aqua',label='$R_3$')
        ax.plot(omega/np.pi,np.real((R1+R2+R3)),color = 'red',ls ='-.',label='Hamming window $R_1+R_2+R_3$')
        ax.legend()
        mmin = min(np.amin(np.real(R1)),np.amin(np.real(R2)),np.amin(np.real(R3)),np.amin(np.real(R1+R2+R3)))
        mmax = N+1
        mmin = mmin - mmin % 1
        ax.set_ylim(mmin,mmax)
        ax.set_yticks(np.arange(mmin-(mmax-mmin)%np.ceil(N/10),mmax,np.ceil(N/10)))
        ax.set_ylabel('A')
    elif mode == 'lin':
        ax.plot(omega/np.pi,np.abs(R1)/N,color='black',label = 'rectangular window')
        #ax.plot(omega/np.pi,np.real(R2),color ='blue', ls='--',label = '$R_2$')
        #ax.plot(omega/np.pi,np.real(R3),color='aqua',label='$R_3$')
        ax.plot(omega/np.pi,np.abs(R1+R2+R3)/N,color = 'red',ls ='-.',label='Hamming window')
        ax.legend()
        ax.set_ylim(-0.1,1.1)
        ax.set_yticks(np.arange(0,1.1,0.1))
        ax.set_ylabel(r'$\frac{|A|}{N}$')
    elif mode == 'log':
        ax.plot(omega/np.pi,20*np.log10(np.abs(R1)/N),color='black',label = 'rectangular window')
        #ax.plot(omega/np.pi,np.real(R2),color ='blue', ls='--',label = '$R_2$')
        #ax.plot(omega/np.pi,np.real(R3),color='aqua',label='$R_3$')
        ax.plot(omega/np.pi,20*np.log10(np.abs(R1+R2+R3)/N),color = 'red',ls ='-.',label='Hamming window')
        ax.legend()
        ax.set_ylim(-60,0)
        ax.set_yticks(np.append(np.arange(-60,-6.1,6),np.arange(-6,0.1,3)))
        ax.set_ylabel(r'$20*\log_{10}\left(\frac{|A|}{N}\right)$')
    if (mode == 'normal' or mode =='lin') and N <=25:
        if N % 2 == 0:
            omega = np.arange(-N//2-1,N//2)*2*np.pi/N
        else:
            omega = np.arange(-N//2,N//2+1)*2*np.pi/N
        omega[N//2+1]=10**(-16)
        ax.plot(omega[1:N//2]/np.pi,np.zeros(len(omega[1:N//2])),ls='none',marker='o',color='red')
        ax.plot([omega[N//2]/np.pi,omega[N//2+2]/np.pi],np.zeros(2),ls='none',marker='o',color='black')
        if mode == 'normal':
            ax.plot(0,N,ls='none',marker='o',color='red')
        else:
            ax.plot(0,1,ls='none',marker='o',color='red')
        ax.plot(omega[N//2+3:]/np.pi,np.zeros(len(omega[N//2+3:])),ls='none',marker='o',color='red')
        ax.plot(np.array([-5/N,+5/N]),np.zeros(2),ls='none',marker='o',color='red')
    ax.set_xlim(-1,1)
    ax.set_xticks(np.arange(-1,1.1,0.2))
    ax.set_title('Hamming window, $N=%d$' %(N))
    ax.grid(True)
    ax.set_xlabel('$\Omega\enspace/\enspace\pi$')
        

In [ ]:
fig, ax = plt.subplots(nrows = 1,ncols=1,figsize=(10,5))
plot_hamming_rect_window(N=16,ax=ax,mode='normal')
fig.tight_layout()
#fig.savefig('graphics/HammingausRectWindow.pdf')

In [ ]:
def plot_hann_hamming_rect_window(N,ax,rect=True,hamming=True,hann=True):
    beta = 21/25
    omega1 = np.arange(-np.pi,0,1/1000*np.pi)
    omega2 = np.arange(0+1/1000*np.pi,np.pi,1/1000*np.pi)
    omega = np.append(np.append(omega1,np.zeros(1)),omega2)
    R11 = np.sin(N*omega1/2)/np.sin(omega1/2)
    R12 = np.sin(N*omega2/2)/np.sin(omega2/2)
    R1 = np.append(np.append(R11,np.array([N])),R12)
    R21 = 1/2*np.sin(N/2*(omega1-2*np.pi/N))/np.sin(1/2*(omega1-2*np.pi/N))
    R22 = 1/2*np.sin(N/2*(omega2-2*np.pi/N))/np.sin(1/2*(omega2-2*np.pi/N))
    #lim N->0 sin(-N/2*2*pi/N)/sin(-1/2*2*pi/N)=lim N->0 sin(-pi)/sin(-pi/n)= L'Hospital lim N->0 -pi*cos(-pi)/(-pi/N*cos(-pi/n))=lim N->0 N*1 = 0
    R2 = np.append(np.append(R21,np.zeros(1)),R22)
    R31 = 1/2*np.sin(N/2*(omega1+2*np.pi/N))/np.sin(1/2*(omega1+2*np.pi/N))
    R32 = 1/2*np.sin(N/2*(omega2+2*np.pi/N))/np.sin(1/2*(omega2+2*np.pi/N))
    #lim N->0 sin(N/2*2*pi/N)/sin(1/2*2*pi/N)=lim N->0 sin(pi)/sin(pi/n)= L'Hospital lim N->0 pi*cos(pi)/(pi/N*cos(pi/n))=lim N->0 N*1 = 0
    R3 = np.append(np.append(R31,np.zeros(1)),R32)
    whann = R1+R2+R3
    whamming = R1+beta*R2+beta*R3
    s = []
    if rect == False and hamming == False and hann == False:
        print("You should print at least one spectra ;)")
        return
    if rect == True:
        ax.plot(omega/np.pi,20*np.log10(np.abs(R1/N)),color='black',label='rectangular window')
        s += ["Rectangular"]
    if hann == True:
        ax.plot(omega/np.pi,20*np.log10(np.abs(whann/N)),ls='--',color='blue',label='Hann window')
        s += ["Hann"]
    if hamming == True:
        ax.plot(omega/np.pi,20*np.log10(np.abs(whamming/N)),color='red',label='Hamming window')
        s += ["Hamming"]
    ax.set_xlim(-1,1)
    ax.set_xticks(np.arange(-1,1.1,0.2))
    ax.set_ylim(-60,0)
    ax.set_yticks(np.append(np.arange(-60,-6.1,6),np.arange(-6,0.1,3)))
    ax.grid(True)
    ax.set_ylabel(r'$20*\log_{10}\left(\frac{|A|}{N}\right)$')
    ax.set_xlabel('$\Omega\enspace/\enspace\pi$')
    string = ""
    for i in range(0,len(s)):
        if i == len(s)-1 and len(s)>1:
            string += "and "+s[i]+" window, $N=%d$"
        elif i == len(s)-1 and len(s) == 1:
            string = s[i]+" window, $N=%d$"
        elif i == 0 and len(s) == 2:
            string+= s[i]+" "
        else:
            string+= s[i] + ", "
    ax.set_title(string %(N))
    ax.legend()

In [ ]:
fig,ax = plt.subplots(1,figsize=(10,5))
plot_hann_hamming_rect_window(16,ax,rect=True,hamming=True,hann=False)
fig.tight_layout()
#fig.savefig('graphics/DTFTHammingWin_log.pdf')
fig,ax = plt.subplots(1,figsize=(10,5))
plot_hann_hamming_rect_window(16,ax,rect=True,hamming=True,hann=True)
fig.tight_layout()
#fig.savefig('graphics/DTFTRectHanningWin_log.pdf')

In [ ]:
-7 - 7 % 4

In [ ]:
x = np.arange(-np.pi,np.pi,1/100)-2*np.pi/16
plt.plot(x,np.sin(x))

In [ ]:
%matplotlib

In [ ]:
N = 16
omega1 = np.arange(-np.pi,0,1/1000*np.pi)
omega2 = np.arange(0+1/1000*np.pi,np.pi,1/1000*np.pi)
omega = np.append(np.append(omega1,np.zeros(1)),omega2)
R11 = np.sin(N*omega1/2)/np.sin(omega1/2)
R12 = np.sin(N*omega2/2)/np.sin(omega2/2)
R1 = np.append(np.append(R11,np.array([N])),R12)
R21 = 1/2*np.sin(N/2*(omega1-2*np.pi/N))/np.sin(1/2*(omega1-2*np.pi/N))
R22 = 1/2*np.sin(N/2*(omega2-2*np.pi/N))/np.sin(1/2*(omega2-2*np.pi/N))
#lim N->0 sin(-N/2*2*pi/N)/sin(-1/2*2*pi/N)=lim N->0 sin(-pi)/sin(-pi/n)= L'Hospital lim N->0 -pi*cos(-pi)/(-pi/N*cos(-pi/n))=lim N->0 N*1 = 0
R2 = np.append(np.append(R21,np.zeros(1)),R22)
R31 = 1/2*np.sin(N/2*(omega1+2*np.pi/N))/np.sin(1/2*(omega1+2*np.pi/N))
R32 = 1/2*np.sin(N/2*(omega2+2*np.pi/N))/np.sin(1/2*(omega2+2*np.pi/N))

In [ ]:
R1 = np.sin(N*omega/2)/(np.sin(omega/2))
plt.plot(R1)
plt.grid(True)
R1 = np.append(np.append(R11,np.array([N])),R12)
plt.plot(np.real(R1))

In [ ]:
15/int(2)

In [ ]:
7//2

In [ ]:
1e-16*2

**Copyright**

The notebooks are provided as [Open Educational Resources](https://en.wikipedia.org/wiki/Open_educational_resources). Feel free to use the notebooks for your own purposes. The text is licensed under [Creative Commons Attribution 4.0](https://creativecommons.org/licenses/by/4.0/), the code of the IPython examples under the [MIT license](https://opensource.org/licenses/MIT). Please attribute the work as follows: *Frank Schultz, Digital Signal Processing - A Tutorial Featuring Computational Examples* with the URL https://github.com/spatialaudio/digital-signal-processing-exercises